In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass  

In [2]:
# step 1
password = getpass.getpass()

bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd
engine = create_engine(connection_string)

In [13]:
# step 2
def rentals_month(engine, month, year):
    query = f"""
    SELECT customer_id, COUNT(*) AS total_rentals
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    GROUP BY customer_id;
    """
    # Execute query and store result in a pandas DataFrame
    df = pd.read_sql(query, engine)
    return df

In [14]:
# step 3
def rental_count_month(df, month, year):
    # Create a new column name for rentals based on month and year
    column_name = f"rentals_{str(month).zfill(2)}_{year}"
    # Rename the 'total_rentals' column to the new column name
    df = df.rename(columns={'total_rentals': column_name})
    return df

In [15]:
# step 4
def compare_rentals(df1, df2):
    # Merge the two DataFrames on 'customer_id', performing an outer join to include all customers from both months
    df_combined = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    
    # Calculate the difference between the two months
    df_combined['difference'] = df_combined.iloc[:, 2] - df_combined.iloc[:, 1]
    
    return df_combined

In [16]:
df_may = rentals_month(engine, 5, 2005)
df_may = rental_count_month(df_may, 5, 2005)

# Get rentals data for June 2005
df_june = rentals_month(engine, 6, 2005)
df_june = rental_count_month(df_june, 6, 2005)

# Compare the two months
df_comparison = compare_rentals(df_may, df_june)

# Display the result
df_comparison.head(20)

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
3,4,0.0,6.0,6.0
4,5,3.0,5.0,2.0
5,6,3.0,4.0,1.0
6,7,5.0,5.0,0.0
7,8,1.0,3.0,2.0
8,9,3.0,2.0,-1.0
9,10,1.0,5.0,4.0
